### Touch

Имеем два набора данных. один набор размечен пользователями 28 примеров, второй набор размечан автоматически 3188.

Проверим, насколько автоматически полученные данные, хорошо описывают размеченные вручную.


In [1]:
import matplotlib.pyplot as plt
import pandas as pn
import numpy as np
%matplotlib inline  


In [2]:
file_name_user = 'data_sets/mphone_mark_users.csv'
file_name_auto = 'data_sets/mphone_mark_auto.csv'

#?pn.read_csv
columns = ["Gender","Fingerprint","TimeUpx","W","H","leng","leng_sum","speed","Force","ForceUp","Rx","Ry","varX","varY","coorXY"]
print('columns: %d' % len(columns))
df_user = pn.read_csv(file_name_user, nrows=15000, names=columns, header=1)
df_auto = pn.read_csv(file_name_auto, nrows=15000, names=columns, header=1)

columns: 15


In [3]:
df_user.describe()

,TimeUpx,W,H,leng,leng_sum,speed,Force,ForceUp,Rx,Ry,varX,varY,coorXY
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,2.700000e+01,24.000000
mean,373.723473,350.370370,662.592593,177.994823,1216.167096,0.483212,0.452393,0.002481,12.715362,12.715362,2086.672672,1.793523e+05,-0.146530
std,302.411272,20.751549,75.541257,296.190706,1043.608911,0.349451,0.499913,0.007526,15.919060,15.919060,3643.059642,4.694765e+05,0.720704
min,0.000000,320.000000,568.000000,24.745730,77.820310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-1.000000
25%,184.715000,320.000000,568.000000,50.585805,386.837215,0.193145,0.000000,0.000000,0.000000,0.000000,113.070085,7.912361e+02,-0.830993
50%,295.000000,360.000000,667.000000,108.321700,720.271370,0.434170,0.056330,0.000000,0.659180,0.659180,538.249140,4.420800e+03,-0.164595
75%,454.083335,360.000000,740.000000,147.380345,2233.540275,0.757295,1.000000,0.000000,28.601315,28.601315,2109.694450,1.430145e+05,0.562660
max,1385.666670,375.000000,760.000000,1591.342210,3439.153620,1.155530,1.000000,0.031670,44.886670,44.886670,14559.183400,2.301841e+06,1.000000


In [6]:
df_auto.describe()

,TimeUpx,W,H,leng,leng_sum,speed,Force,ForceUp,Rx,Ry,varX,varY
count,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000
mean,771.764652,368.568346,639.807897,138.054755,2264.775905,0.359535,0.803830,0.070930,2.168585,2.204334,1788.098974,3122.489911
std,1171.353036,77.293368,68.062270,73.612584,2623.534999,0.298511,0.365201,0.252262,3.192029,3.179487,6511.340500,4394.646649
min,0.000000,320.000000,320.000000,0.000000,0.000000,0.000000,0.003930,0.000000,0.000000,0.000000,0.000000,0.000000
25%,243.895837,360.000000,640.000000,83.697188,856.507152,0.205483,1.000000,0.000000,0.000000,0.000000,111.687115,562.264833
50%,460.525645,360.000000,640.000000,120.338900,1535.639780,0.318375,1.000000,0.000000,0.708340,0.793565,343.992190,1638.736115
75%,854.645243,360.000000,640.000000,174.479582,2638.827905,0.454695,1.000000,0.000000,3.291508,3.291508,1108.977437,3897.983607
max,20662.000000,1280.000000,1280.000000,584.874850,34518.969310,10.135830,1.091670,1.000000,22.575510,22.575510,198307.582230,51175.770670


In [5]:
from sklearn.preprocessing import MinMaxScaler


df_user = df_user.fillna(0)
df_auto = df_auto.fillna(0)

def map_target(cl):
    if(cl == 'f'):
        return 0
    return 1

X = df_auto[columns[2:]]
X_test =df_user[columns[2:]]
Y = df_auto['Gender'].map(map_target)
Y_test = df_user['Gender'].map(map_target) 

scaler = MinMaxScaler()
X_std = scaler.fit_transform(X)
X_test_std = scaler.transform(X_test)

#### RandomForestClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=20, max_depth=4, random_state=0)
clf.fit(X_std,Y)
print(clf.score(X_test_std, Y_test))

0.6296296296296297


In [7]:
print classification_report(clf.predict(X_test_std), Y_test)

             precision    recall  f1-score   support

          0       0.22      0.40      0.29         5
          1       0.83      0.68      0.75        22

avg / total       0.72      0.63      0.66        27



#### GradientBoostingClassifier

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(max_depth=2, n_estimators=25, random_state=42)
clf.fit(X_std,Y)
print(clf.score(X_test_std, Y_test))

0.6666666666666666


In [33]:
print classification_report(clf.predict(X_test_std), Y_test)

             precision    recall  f1-score   support

          0       0.44      0.44      0.44         9
          1       0.72      0.72      0.72        18

avg / total       0.63      0.63      0.63        27



In [39]:
from sklearn.model_selection import cross_val_score
#?GradientBoostingClassifier
for md in [2,3,4,5,6]:
    for ne in [10,25,50,100,200]:
        clf = GradientBoostingClassifier(max_depth=md,n_estimators=ne, random_state=42)
        clf.fit(X_std,Y)
        sc = clf.score(X_test_std, Y_test)
        scores = cross_val_score(clf,  X_test_std, Y_test, cv=5)
        print("max_depth: %d, n_estimators: %3d, score: %.3f, m-score: %.3f, sc: [%s]" % 
              (md, ne, sc, scores.mean(), ', '.join(map(lambda t: '%.3f' % t,scores.tolist()))))
        

max_depth: 2, n_estimators:  10, score: 0.667, m-score: 0.693, sc: [0.667, 0.667, 0.833, 0.800, 0.500]
max_depth: 2, n_estimators:  25, score: 0.667, m-score: 0.693, sc: [0.667, 0.667, 0.833, 0.800, 0.500]
max_depth: 2, n_estimators:  50, score: 0.667, m-score: 0.653, sc: [0.667, 0.667, 0.833, 0.600, 0.500]
max_depth: 2, n_estimators: 100, score: 0.593, m-score: 0.653, sc: [0.833, 0.667, 0.667, 0.600, 0.500]
max_depth: 2, n_estimators: 200, score: 0.481, m-score: 0.743, sc: [0.833, 0.667, 0.667, 0.800, 0.750]
max_depth: 3, n_estimators:  10, score: 0.630, m-score: 0.693, sc: [0.667, 0.667, 0.833, 0.800, 0.500]
max_depth: 3, n_estimators:  25, score: 0.630, m-score: 0.727, sc: [0.833, 0.667, 0.833, 0.800, 0.500]
max_depth: 3, n_estimators:  50, score: 0.630, m-score: 0.693, sc: [0.833, 0.667, 0.667, 0.800, 0.500]
max_depth: 3, n_estimators: 100, score: 0.630, m-score: 0.727, sc: [0.833, 0.667, 0.833, 0.800, 0.500]
max_depth: 3, n_estimators: 200, score: 0.630, m-score: 0.727, sc: [0.833

In [37]:
?GradientBoostingClassifier

####  NN keras

In [15]:
from keras import Sequential
from keras.layers import Dense

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
classifier = Sequential() 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=len(X_std[0]))) 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [25]:
classifier.fit(X_std,Y, batch_size=10, epochs=100,validation_data=(X_test_std, Y_test))

Train on 4713 samples, validate on 27 samples
Epoch 1/100
4713/4713 [==============================] - 1s 222us/step - loss: 0.6526 - acc: 0.6287 - val_loss: 1.6151 - val_acc: 0.5185
Epoch 2/100
4713/4713 [==============================] - 1s 190us/step - loss: 0.6136 - acc: 0.6722 - val_loss: 1.6351 - val_acc: 0.4444
Epoch 3/100
4713/4713 [==============================] - 1s 189us/step - loss: 0.5711 - acc: 0.6970 - val_loss: 2.0860 - val_acc: 0.4074
Epoch 4/100
4713/4713 [==============================] - 1s 187us/step - loss: 0.5549 - acc: 0.7066 - val_loss: 2.2888 - val_acc: 0.4074
Epoch 5/100
4713/4713 [==============================] - 1s 205us/step - loss: 0.5436 - acc: 0.7159 - val_loss: 2.2843 - val_acc: 0.4074
Epoch 6/100
4713/4713 [==============================] - 1s 194us/step - loss: 0.5348 - acc: 0.7223 - val_loss: 2.3146 - val_acc: 0.4444
Epoch 7/100
4713/4713 [==============================] - 1s 187us/step - loss: 0.5277 - acc: 0.7269 - val_loss: 2.3760 - val_acc: 0.

4713/4713 [==============================] - 1s 189us/step - loss: 0.3509 - acc: 0.8445 - val_loss: 3.8078 - val_acc: 0.5926
Epoch 61/100
4713/4713 [==============================] - 1s 197us/step - loss: 0.3506 - acc: 0.8387 - val_loss: 3.7198 - val_acc: 0.6296
Epoch 62/100
4713/4713 [==============================] - 1s 193us/step - loss: 0.3464 - acc: 0.8411 - val_loss: 3.6254 - val_acc: 0.5926
Epoch 63/100
4713/4713 [==============================] - 1s 187us/step - loss: 0.3415 - acc: 0.8483 - val_loss: 3.8810 - val_acc: 0.4815
Epoch 64/100
4713/4713 [==============================] - 1s 186us/step - loss: 0.3442 - acc: 0.8474 - val_loss: 3.6917 - val_acc: 0.5926
Epoch 65/100
4713/4713 [==============================] - 1s 192us/step - loss: 0.3434 - acc: 0.8449 - val_loss: 3.8086 - val_acc: 0.5556
Epoch 66/100
4713/4713 [==============================] - 1s 191us/step - loss: 0.3397 - acc: 0.8462 - val_loss: 4.1213 - val_acc: 0.5556
Epoch 67/100
4713/4713 [=======================

In [26]:
y_pred= classifier.predict(X_test_std)
y_pred =(y_pred>0.5)
print classification_report(y_pred, Y_test)

             precision    recall  f1-score   support

      False       0.56      0.38      0.45        13
       True       0.56      0.71      0.63        14

avg / total       0.56      0.56      0.54        27



### Split train data

In [27]:
from sklearn.model_selection import train_test_split

X_train_std_, X_test_std_, y_train_std_, y_test_std_ = train_test_split(X_std, Y, test_size=0.33, random_state=42)
print(len(X_train_std_))

3157


### GradientBoostingClassifier

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(max_depth=4, n_estimators=100, random_state=42)
clf.fit(X_train_std_,y_train_std_)
print(clf.score(X_test_std_, y_test_std_))
#print(clf.score(X_test_std, Y_test))

0.8245501285347043
0.6296296296296297


In [30]:
from sklearn.model_selection import cross_val_score
#?GradientBoostingClassifier
for md in [2,3,4,5,6]:
    for ne in [10,25,50,100,200]:
        clf = GradientBoostingClassifier(max_depth=md,n_estimators=ne, random_state=42)
        clf.fit(X_train_std_,y_train_std_)
        scores = cross_val_score(clf,  X_test_std_, y_test_std_, cv=5)
        print("max_depth: %d, n_estimators: %3d, score: %.3f, sc: [%s]" % 
              (md, ne, clf.score(X_test_std_, y_test_std_), ', '.join(map(lambda t: '%.3f' % t,scores.tolist()))))

max_depth: 2, n_estimators:  10, score: 0.714, sc: [0.705, 0.717, 0.698, 0.720, 0.730]
max_depth: 2, n_estimators:  25, score: 0.759, sc: [0.712, 0.749, 0.695, 0.704, 0.752]
max_depth: 2, n_estimators:  50, score: 0.772, sc: [0.744, 0.765, 0.746, 0.762, 0.765]
max_depth: 2, n_estimators: 100, score: 0.790, sc: [0.769, 0.778, 0.778, 0.768, 0.775]
max_depth: 2, n_estimators: 200, score: 0.808, sc: [0.779, 0.765, 0.804, 0.791, 0.775]
max_depth: 3, n_estimators:  10, score: 0.751, sc: [0.715, 0.736, 0.749, 0.733, 0.733]
max_depth: 3, n_estimators:  25, score: 0.805, sc: [0.792, 0.772, 0.804, 0.781, 0.791]
max_depth: 3, n_estimators:  50, score: 0.810, sc: [0.779, 0.788, 0.820, 0.807, 0.785]
max_depth: 3, n_estimators: 100, score: 0.816, sc: [0.798, 0.788, 0.810, 0.810, 0.791]
max_depth: 3, n_estimators: 200, score: 0.823, sc: [0.785, 0.794, 0.801, 0.794, 0.781]
max_depth: 4, n_estimators:  10, score: 0.760, sc: [0.769, 0.762, 0.756, 0.759, 0.749]
max_depth: 4, n_estimators:  25, score: 0.8

### NN Keras

In [ ]:
from keras import Sequential
from keras.layers import Dense

In [27]:
classifier = Sequential() 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=len(X_std[0]))) 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [33]:
classifier.fit(X_train_std_,y_train_std_, batch_size=10, epochs=10,validation_data=(X_test_std_, y_test_std_))

Train on 2135 samples, validate on 1053 samples
Epoch 1/10
2135/2135 [==============================] - 1s 235us/step - loss: 0.3964 - acc: 0.8187 - val_loss: 0.4506 - val_acc: 0.8053
Epoch 2/10
2135/2135 [==============================] - 0s 213us/step - loss: 0.3863 - acc: 0.8267 - val_loss: 0.4544 - val_acc: 0.8025
Epoch 3/10
2135/2135 [==============================] - 1s 244us/step - loss: 0.3906 - acc: 0.8197 - val_loss: 0.4482 - val_acc: 0.7987
Epoch 4/10
2135/2135 [==============================] - 0s 220us/step - loss: 0.3915 - acc: 0.8262 - val_loss: 0.5045 - val_acc: 0.7540
Epoch 5/10
2135/2135 [==============================] - 0s 217us/step - loss: 0.3849 - acc: 0.8286 - val_loss: 0.4521 - val_acc: 0.7958
Epoch 6/10
2135/2135 [==============================] - 0s 221us/step - loss: 0.3774 - acc: 0.8253 - val_loss: 0.4620 - val_acc: 0.8063
Epoch 7/10
2135/2135 [==============================] - 0s 217us/step - loss: 0.3751 - acc: 0.8248 - val_loss: 0.4743 - val_acc: 0.7806


In [23]:
y_pred= classifier.predict(X_test_std_)
y_pred =(y_pred>0.5)
print classification_report(y_pred, y_test_std_)

NameError: name 'X_test_std_' is not defined